In [ ]:
import os
import subprocess

dataset = []

path= 'images'
languages = ['c', 'asm1', 'asm2']
filters = ['hsl', 'blur', 'merge']
command = {
    'blur': './runTests {language} {filter} {src} {dst}',
    'merge': './runTests {language} {filter} {src} {src2} {dst} {alpha}',
    'hsl': './runTests {language} {filter} {src} {dst} {h} {s} {l}'
}

In [ ]:
hh = [0.0, 50.93, -30.72, 181.44]
ss = [0.0, 0.313, -0.409, -0.843]
ll = [0.0, -0.131, 0.213, 0.713]
alpha = [0, 0.131, 0.223, 0.313, 0.409, 0.5, 0.662, 0.713, 0.843, 0.921, 1.0]
dataset = []

def worker(filt, lang, l):
    output = []
    rpath = os.path.join(path, filt, lang)

    for img in os.listdir(rpath):
        if os.path.isfile(img):
            if filt == 'hsl':
                for (h, s, l) in itertools.product(hh, ss, ll):
                    comm = command[filt].format(
                            language = language,
                            filter = filt,
                            src = os.path.join('./images', image),
                            dst = './current.bmp',
                            h = h,
                            s = s,
                            l = l
                        )

                    delta = subprocess.check_output(comm, shell=True)
                    delta = delta.strip()

                    output.append({
                        'filename': img
                        'language': language,
                        'filter': filt,
                        'delta': delta,
                        'h': h,
                        's': s,
                        'l': l
                    })
            elif filt == 'blur':
                comm = command[filt].format(
                        language = language,
                        filter = filt,
                        src = os.path.join('./images', image),
                        dst = './current.bmp'
                    )

                delta = subprocess.check_output(comm, shell=True)
                delta = delta.strip()

                output.append({
                    'filename': img
                    'language': language,
                    'filter': filt,
                    'delta': delta,
                })
            elif filt == 'merge':
                for a in alpha:
                    comm = command[filt].format(
                            language = language,
                            filter = filt,
                            src = os.path.join('./images', image),
                            src2 = os.path.join('./images', image),
                            dst = './current.bmp',
                            value = a
                        )

                    delta = subprocess.check_output(comm, shell=True)
                    delta = delta.strip()

                    output.append({
                        'filename': img,
                        'language': language,
                        'filter': filt,
                        'delta': delta,
                        'alpha': a
                    })
                    
    l.acquire()
    dataset += output
    l.release()
                    
jobs = []
l = Lock()
for filt, lang in itertools.product(filters, languages):
    p = multiprocessing.Process(target=worker, args=(filt, lang, l))
    jobs.append(p)
    p.start()

for job in jobs:
    job.join()
    
dataset = pd.DataFrame(dataset)